Save Gdrive



In [4]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import sys
#gatien
#my_local_drive='/content/gdrive/MyDrive/M1/Projet/donnees'
#gaby
my_local_drive='/content/gdrive/My Drive/M1_IASD/Colab Notebooks/ML/Projet/donnees/'
sys.path.append(my_local_drive)


# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd
%ls

/content/gdrive/My Drive/M1_IASD/Colab Notebooks/ML/Projet/donnees
HAI817_Projet_test.csv   HAI817_Projet_train.csv  Projet_HAI817I.ipynb
HAI817_Projet_test.xlsx  MyNLPUtilities.py


## **Travaux pratiques**

Importer les différentes choses qu'on a besoin

In [6]:
# Librairie a installer

!pip install langdetect 
!pip install contractions

# Importation des différentes librairies utiles pour le notebook

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import pandas as pd
import numpy as np 
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


# Importation des différentes librairies utiles pour le notebook ingenieurie logicielle des données textuelles

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle 
from scipy.stats import randint
import string
import time
import base64
import re

import contractions

# librairie BeautifulSoup
from bs4 import BeautifulSoup

# librairie affichage
import wordcloud

## detection de language
import langdetect 

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import RegexpParser
# il est possible de charger l'ensemble des librairies en une seule fois 
# décocher le commentaire de la ligne ci-dessous
#nltk.download('all') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords

import spacy
from spacy.tokens import Span
from spacy.lang.fr import French

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=1336f66d2b764e6f488393197f1340b192c86fe69516daf89360fd1ac012ef65
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 14.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


On récupère les infos du dataset

In [7]:
df=pd.read_csv('HAI817_Projet_test.csv')

Afficher le nombre de ligne et de colonnes du dataframe ainsi que les 5 premières lignes

In [8]:
print(f"Nombre de lignes : {df.shape[0]}\nNombre de colonnes : {df.shape[1]}")

df.head(5)

Nombre de lignes : 612
Nombre de colonnes : 4


,ID,text,title,our rating
0,122653045997905671927713471889615536378,The deputy secretary at the US Treasury has sa...,US Treasury deputy sec warns that shortages li...,mixture
1,275389285957305997321446227088442471741,KABUL—Approximately twelve minutes after U.S. ...,CNN Praises Taliban For Wearing Masks During A...,other
2,333248764296609831067233855420575814716,The vast majority of (oblivious) Americans hav...,Tennessee Has Just LEGALIZED Government COVID ...,false
3,264019763253447756851916399533799891538,(Natural News) There’s a secret layer of infor...,MEDICAL SHOCKER: Scientists at Sloan Kettering...,false
4,158073737187690682830899773280916034317,"A recent study reported by the NCBI, which is ...",Study Results: Facemasks are Ineffective to Bl...,false


# Pré-traitement

Copier le df dans un autre pour le pré-traitement

In [25]:
dfPret = df.copy()
#display(dfPret)

,ID,text,title,our rating
0,122653045997905671927713471889615536378,The deputy secretary at the US Treasury has sa...,US Treasury deputy sec warns that shortages li...,mixture
1,275389285957305997321446227088442471741,KABUL—Approximately twelve minutes after U.S. ...,CNN Praises Taliban For Wearing Masks During A...,other
2,333248764296609831067233855420575814716,The vast majority of (oblivious) Americans hav...,Tennessee Has Just LEGALIZED Government COVID ...,false
3,264019763253447756851916399533799891538,(Natural News) There’s a secret layer of infor...,MEDICAL SHOCKER: Scientists at Sloan Kettering...,false
4,158073737187690682830899773280916034317,"A recent study reported by the NCBI, which is ...",Study Results: Facemasks are Ineffective to Bl...,false
...,...,...,...,...
607,206921579192720402152220292846254137866,DISTRIBUTE TO EVERYONE who does not want to be...,"‘From October 20, the Covid pass and vaccinati...",false
608,26464356958085668535128110139955868781,The physician who served as Bill Gates’ privat...,Bill Gates' Former Doctor Says He 'Refused To ...,other
609,189700315351432315169761694613833047445,"A customer leaves an SAQ outlet on Jan. 6, the...",Quebec’s expansion of COVID-19 vaccine passpor...,true
610,188373492569049166266633359075598402208,Vaccine coercion activists are using the fake ...,ALERT: Luc Montagnier Did NOT Say Vaccine Woul...,false


Recuperer les titres et les text

In [36]:
print("Liste des titres")
titres = dfPret["title"]
display(titres)

print("\n\nListe des textes")
textes = dfPret["text"]
display(textes)

Liste des titres


0      US Treasury deputy sec warns that shortages li...
1      CNN Praises Taliban For Wearing Masks During A...
2      Tennessee Has Just LEGALIZED Government COVID ...
3      MEDICAL SHOCKER: Scientists at Sloan Kettering...
4      Study Results: Facemasks are Ineffective to Bl...
                             ...                        
607    ‘From October 20, the Covid pass and vaccinati...
608    Bill Gates' Former Doctor Says He 'Refused To ...
609    Quebec’s expansion of COVID-19 vaccine passpor...
610    ALERT: Luc Montagnier Did NOT Say Vaccine Woul...
611    Quebec’s cannabis, liquor stores to require co...
Name: title, Length: 612, dtype: object



Liste des textes


0      The deputy secretary at the US Treasury has sa...
1      KABUL—Approximately twelve minutes after U.S. ...
2      The vast majority of (oblivious) Americans hav...
3      (Natural News) There’s a secret layer of infor...
4      A recent study reported by the NCBI, which is ...
                             ...                        
607    DISTRIBUTE TO EVERYONE who does not want to be...
608    The physician who served as Bill Gates’ privat...
609    A customer leaves an SAQ outlet on Jan. 6, the...
610    Vaccine coercion activists are using the fake ...
611    Placeholder while article actions load  Quebec...
Name: text, Length: 612, dtype: object

Supprimer les caractères spéciaux

In [49]:
print("TITRES")
for i in range(len(titres)) :
  # suppression des caractères spéciaux
  sentence = re.sub(r'[^\w\s]',' ', str(titres[i]))
  titres[i] = sentence
display(titres)
print("\n\n")

print("TEXTES")
for i in range(len(textes)) :
  # suppression des caractères spéciaux
  sentence = re.sub(r'[^\w\s]',' ', str(textes[i]))
  textes[i] = sentence
display(textes)

TITRES


0      US Treasury deputy sec warns that shortages li...
1      CNN Praises Taliban For Wearing Masks During A...
2      Tennessee Has Just LEGALIZED Government COVID ...
3      MEDICAL SHOCKER  Scientists at Sloan Kettering...
4      Study Results  Facemasks are Ineffective to Bl...
                             ...                        
607     From October 20  the Covid pass and vaccinati...
608    Bill Gates  Former Doctor Says He  Refused To ...
609    Quebec expansion of COVID 19 vaccine passports...
610    ALERT  Luc Montagnier Did NOT Say Vaccine Woul...
611    Quebec cannabis  liquor stores to require coro...
Name: title, Length: 612, dtype: object




TEXTES


0      The deputy secretary at the US Treasury has sa...
1      KABUL Approximately twelve minutes after troop...
2      The vast majority of  oblivious  Americans hav...
3       Natural News  There secret layer of informati...
4      A recent study reported by the NCBI  which is ...
                             ...                        
607    DISTRIBUTE TO EVERYONE who does not want to be...
608    The physician who served as Bill Gates  privat...
609    A customer leaves an SAQ outlet on Jan  6  the...
610    Vaccine coercion activists are using the fake ...
611    Placeholder while article actions load  Quebec...
Name: text, Length: 612, dtype: object

Tokenisation

In [61]:
word_tokens_list = []
for i in range(len(titres)):
    # tokenisation
    word_tokens = word_tokenize(titres[i])
    #print("Premiers tokens après nettoyage des données ", word_tokens)
    word_tokens_list.append(word_tokens)
print(word_tokens_list)

[['US', 'Treasury', 'deputy', 'sec', 'warns', 'that', 'shortages', 'likely', 'to', 'continue', 'until', 'EVERYONE', 'is', 'jabbed'], ['CNN', 'Praises', 'Taliban', 'For', 'Wearing', 'Masks', 'During', 'Attack'], ['Tennessee', 'Has', 'Just', 'LEGALIZED', 'Government', 'COVID', 'Kidnappings', 'The', 'Daily', 'Allegiant'], ['MEDICAL', 'SHOCKER', 'Scientists', 'at', 'Sloan', 'Kettering', 'discover', 'mRNA', 'inactivates', 'tumor', 'suppressing', 'proteins', 'meaning', 'it', 'can', 'promote', 'cancer'], ['Study', 'Results', 'Facemasks', 'are', 'Ineffective', 'to', 'Block', 'Transmission', 'of', 'COVID', '19', 'and', 'Actually', 'Can', 'Cause', 'Health', 'Deterioration', 'and', 'Premature', 'Death'], ['How', 'to', 'Neutralize', 'Potential', 'Damage', 'from', 'mRNA', 'Vaccines'], ['CDC', 'Quietly', 'Admits', 'The', 'Death', 'Toll', 'From', 'COVID', 'Vaccines', 'Is', 'Greater', 'Than', 'Every', 'Vaccine', 'In', 'The', 'Last', '20', 'Years', 'COMBINED'], ['EXCLUSIVE', 'Per', 'the', 'CDC', 'There

Mettre en minuscule

In [63]:
"""min_word_tokens = []
for i in range (len(word_tokens_list)) :
    #Mettre en minuscule
    mots_minuscules = []
    for mot in word_tokens_list[i]:
        mots_minuscules.append(mot.lower())
    min_word_tokens.append(mots_minuscules)
print(min_word_tokens)"""

[['us', 'treasury', 'deputy', 'sec', 'warns', 'that', 'shortages', 'likely', 'to', 'continue', 'until', 'everyone', 'is', 'jabbed'], ['cnn', 'praises', 'taliban', 'for', 'wearing', 'masks', 'during', 'attack'], ['tennessee', 'has', 'just', 'legalized', 'government', 'covid', 'kidnappings', 'the', 'daily', 'allegiant'], ['medical', 'shocker', 'scientists', 'at', 'sloan', 'kettering', 'discover', 'mrna', 'inactivates', 'tumor', 'suppressing', 'proteins', 'meaning', 'it', 'can', 'promote', 'cancer'], ['study', 'results', 'facemasks', 'are', 'ineffective', 'to', 'block', 'transmission', 'of', 'covid', '19', 'and', 'actually', 'can', 'cause', 'health', 'deterioration', 'and', 'premature', 'death'], ['how', 'to', 'neutralize', 'potential', 'damage', 'from', 'mrna', 'vaccines'], ['cdc', 'quietly', 'admits', 'the', 'death', 'toll', 'from', 'covid', 'vaccines', 'is', 'greater', 'than', 'every', 'vaccine', 'in', 'the', 'last', '20', 'years', 'combined'], ['exclusive', 'per', 'the', 'cdc', 'there

In [77]:
cate_word_tokens = []
for i in range (len(word_tokens_list)) :
  # ne retenir que les categories NN et VB et JJ
  # sauvegarde des mots dans un tableau selective_words. 
  # selective_pos_words contient le mot et son tag
  pos = nltk.pos_tag(word_tokens_list[i])
  selective_pos = ['NNP', 'NN','VB', 'JJ', 'RB', 'PR']
  selective_pos_words = []
  selective_words=[]
  for word,tag in pos:
      if tag in selective_pos:
          selective_pos_words.append((word,tag))
          selective_words.append(word)

  cate_word_tokens.append(selective_pos_words)

print ("Tokens conservés de catégorie NNP ou NN ou VB ou 'JJ' ou 'RB' ou 'PR' :")
print(cate_word_tokens)


Tokens conservés de catégorie NNP ou NN ou VB ou 'JJ' ou 'RB' ou 'PR' :
[[('US', 'NNP'), ('Treasury', 'NNP'), ('deputy', 'NN'), ('sec', 'NN'), ('likely', 'RB'), ('continue', 'VB'), ('EVERYONE', 'NNP')], [('CNN', 'NNP'), ('Praises', 'NNP'), ('Taliban', 'NNP'), ('Wearing', 'NNP'), ('Masks', 'NNP'), ('Attack', 'NNP')], [('Tennessee', 'NNP'), ('Has', 'NNP'), ('Just', 'NNP'), ('LEGALIZED', 'NNP'), ('Government', 'NNP'), ('COVID', 'NNP'), ('Kidnappings', 'NNP'), ('Daily', 'NNP'), ('Allegiant', 'NNP')], [('MEDICAL', 'NNP'), ('SHOCKER', 'NNP'), ('Sloan', 'NNP'), ('Kettering', 'NNP'), ('discover', 'NN'), ('mRNA', 'NN'), ('promote', 'VB'), ('cancer', 'NN')], [('Study', 'NNP'), ('Results', 'NNP'), ('Facemasks', 'NNP'), ('Ineffective', 'NNP'), ('Block', 'NNP'), ('Transmission', 'NNP'), ('COVID', 'NNP'), ('Actually', 'RB'), ('Can', 'NNP'), ('Cause', 'NNP'), ('Health', 'NNP'), ('Deterioration', 'NNP'), ('Premature', 'NNP'), ('Death', 'NNP')], [('Neutralize', 'VB'), ('Potential', 'JJ'), ('Damage', 'N

In [71]:
import spacy

nlp = spacy.load('en_core_web_sm')

text = "US est le pays de l'Amérique. Us Est Le Pays De L'Amérique."

doc = nlp(text)

for token in doc:
    if token.is_title and not token.is_stop:
        print(token.text, "est un vrai nom propre")
    else:
        print(token.text, "n'est pas un vrai nom propre")


US n'est pas un vrai nom propre
est n'est pas un vrai nom propre
le n'est pas un vrai nom propre
pays n'est pas un vrai nom propre
de n'est pas un vrai nom propre
l'Amérique n'est pas un vrai nom propre
. n'est pas un vrai nom propre
Us n'est pas un vrai nom propre
Est est un vrai nom propre
Le est un vrai nom propre
Pays est un vrai nom propre
De est un vrai nom propre
L'Amérique est un vrai nom propre
. n'est pas un vrai nom propre


In [82]:
for word_tokens in word_tokens_list :
  # Racinisation
  ps=nltk.stem.porter.PorterStemmer()

  print(f"Racinisation : {[ps.stem(word) for word in word_tokens]}")

Racinisation : ['us', 'treasuri', 'deputi', 'sec', 'warn', 'that', 'shortag', 'like', 'to', 'continu', 'until', 'everyon', 'is', 'jab']
Racinisation : ['cnn', 'prais', 'taliban', 'for', 'wear', 'mask', 'dure', 'attack']
Racinisation : ['tennesse', 'ha', 'just', 'legal', 'govern', 'covid', 'kidnap', 'the', 'daili', 'allegi']
Racinisation : ['medic', 'shocker', 'scientist', 'at', 'sloan', 'ketter', 'discov', 'mrna', 'inactiv', 'tumor', 'suppress', 'protein', 'mean', 'it', 'can', 'promot', 'cancer']
Racinisation : ['studi', 'result', 'facemask', 'are', 'ineffect', 'to', 'block', 'transmiss', 'of', 'covid', '19', 'and', 'actual', 'can', 'caus', 'health', 'deterior', 'and', 'prematur', 'death']
Racinisation : ['how', 'to', 'neutral', 'potenti', 'damag', 'from', 'mrna', 'vaccin']
Racinisation : ['cdc', 'quietli', 'admit', 'the', 'death', 'toll', 'from', 'covid', 'vaccin', 'is', 'greater', 'than', 'everi', 'vaccin', 'in', 'the', 'last', '20', 'year', 'combin']
Racinisation : ['exclus', 'per',

Suppresion des stops worlds

In [73]:
nltk.download("stopwords")
from nltk.corpus import stopwords
the_stopwords=set(stopwords.words("english"))
print (the_stopwords)

{'itself', 'more', 'during', 'few', 'were', 'd', 'had', "shan't", 'couldn', 'ma', "you've", 'your', "it's", 'mustn', 'such', 'not', 'yourselves', 'he', 'nor', 'won', 'between', 'these', 'on', 'but', 'only', "she's", 'him', 're', "didn't", 'you', 'isn', 'that', 'under', 'some', 'after', 'hasn', 'each', 'their', 'because', 'does', 'there', 'yourself', 'm', 'below', 'than', 'll', 'did', 'once', 'we', 'y', 'how', 'are', 'ain', "couldn't", 'our', 'both', 'whom', "hadn't", 'mightn', 'with', 'other', 'they', 'at', 'it', 'shan', 'while', 'further', "aren't", 've', 'to', 'in', 'her', 'herself', 'was', 'before', 'hers', "that'll", 'himself', 'through', "doesn't", 'just', 'being', 'hadn', "wasn't", 'over', 't', "haven't", "needn't", 'when', 'most', 'what', 'ours', 'i', 'off', 'themselves', 'into', 'from', 'don', 'very', 'been', "hasn't", 'shouldn', 'wasn', 'or', 'now', 'have', 'then', "mustn't", 'she', 'needn', "you'd", 'all', 'his', 'me', "wouldn't", 'no', 'o', 'until', 'why', 'aren', 'be', 'for

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
for word_tokens in word_tokens_list :
  print ("Avant suppression des stopwords")
  print (word_tokens)
  tokens_Alice=[word for word in word_tokens if word not in the_stopwords]
  print ("Après suppression des stopwords")
  print (tokens_Alice)
  print("\n\n")

Avant suppression des stopwords
['US', 'Treasury', 'deputy', 'sec', 'warns', 'that', 'shortages', 'likely', 'to', 'continue', 'until', 'EVERYONE', 'is', 'jabbed']
Après suppression des stopwords
['US', 'Treasury', 'deputy', 'sec', 'warns', 'shortages', 'likely', 'continue', 'EVERYONE', 'jabbed']



Avant suppression des stopwords
['CNN', 'Praises', 'Taliban', 'For', 'Wearing', 'Masks', 'During', 'Attack']
Après suppression des stopwords
['CNN', 'Praises', 'Taliban', 'For', 'Wearing', 'Masks', 'During', 'Attack']



Avant suppression des stopwords
['Tennessee', 'Has', 'Just', 'LEGALIZED', 'Government', 'COVID', 'Kidnappings', 'The', 'Daily', 'Allegiant']
Après suppression des stopwords
['Tennessee', 'Has', 'Just', 'LEGALIZED', 'Government', 'COVID', 'Kidnappings', 'The', 'Daily', 'Allegiant']



Avant suppression des stopwords
['MEDICAL', 'SHOCKER', 'Scientists', 'at', 'Sloan', 'Kettering', 'discover', 'mRNA', 'inactivates', 'tumor', 'suppressing', 'proteins', 'meaning', 'it', 'can', 'p

# Classification

ID                                       text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

Recuperer les valeurs VRAI, FAUX et AUTRE et les MIXTES, VRAI ou FAUX

In [29]:
total_vrai = df.loc[df["our rating"] == "true"]
display(f"Nombre de lignes pour total_vrai : {total_vrai.shape[0]}")

total_faux = df.loc[df["our rating"] == "false"]
display(f"Nombre de lignes pour total_faux : {total_faux.shape[0]}")

total_vrai_ou_faux = df.loc[(df["our rating"] == "false") | (df["our rating"] == "true")]
display(f"Nombre de lignes pour total_vrai_ou_faux : {total_vrai_ou_faux.shape[0]}")

total_other = df.loc[df["our rating"] == "other"]
display(f"Nombre de lignes pour total_other : {total_other.shape[0]}")

total_mixte = df.loc[df["our rating"] == "mixture"]
display(f"Nombre de lignes pour total_mixture : {total_mixte.shape[0]}")

Output hidden; open in https://colab.research.google.com to view.

Il est important d'analyser les histogrammes de chaque variable pour mieux comprendre comment les données sont réparties.  
A l'aide du code suivant, afficher les différents histogrammes.  
import matplotlib.pyplot as plt  
df.hist(bins=50, figsize=(20, 15))  
plt.show()  

In [ ]:
df.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
sns.heatmap(df.isnull(), cbar=False)

Existe-t'il des valeurs nulles ? Existe-til des valeurs manquantes ? Rappel vous pouvez le voir avec des histogrammes mais aussi avec une heatmap.  

En fait on peut constater qu'il n'y a pas de valeurs manquantes avec le heatmap mais par contre il y a des valeurs nulles. Il faut toujours faire attention à la manière dont sont codées les valeurs manquantes. Ici nous voyons dans les histogrammes que pour BMI, BloodP, PlGlcConc, SkinThick, TwoHourSerIns il existe des valeurs manquantes. Le nombre de grossesses n'est pas considéré comme une valeur manquante bien sûr.  

Transformer les valeurs nulles par la médiane de la série. 

In [ ]:
# valeur médiane
median_bmi = df['BMI'].median()
# remplacement par la médiane
df['BMI'] = df['BMI'].replace(
 to_replace=0, value=median_bmi)

# valeur médiane
median_bloodP = df['BloodP'].median()
# remplacement par la médiane
df['BloodP'] = df['BloodP'].replace(
 to_replace=0, value=median_bloodP)

# valeur médiane
median_PlGlcConc = df['PlGlcConc'].median()
# remplacement par la médiane
df['PlGlcConc'] = df['PlGlcConc'].replace(
 to_replace=0, value=median_PlGlcConc)

# valeur médiane
median_SkinThick = df['SkinThick'].median()
# remplacement par la médiane
df['SkinThick'] = df['SkinThick'].replace(
 to_replace=0, value=median_SkinThick)

# valeur médiane
median_TwoHourSerIns = df['TwoHourSerIns'].median()
# remplacement par la médiane
df['TwoHourSerIns'] = df['TwoHourSerIns'].replace(
 to_replace=0, value=median_TwoHourSerIns)

Les données sont, à présent, transformées et nous allons pouvoir créer un jeu de données de test et d'apprentissage. Faire une copie du dataframe en df2. Sur df appliquer un scaling pour normaliser les valeurs par rapport à la moyenne et l'écart type (utilisation de StandardScaler (). Nous conservons la copie df2 sans transformation.   
L'objectif à présent est d'appliquer différents classifieurs pour voir celui qui est le plus performant. Pour le ou les meilleurs rechercher les hyperparamètres et créer un pipeline à sauvegarder. Il faut ensuite pouvoir traiter de nouvelles données pour prédire si il y a diabète ou pas.  

Tester les résultats sur df et sur df2. 

In [ ]:
# pour essayer sans scaling
df2=df.copy()

#traitement de df
# séparation données à prédire
array = df2.values
X = array[:,0:8]
y = array[:,8]

#utilisation de StandardScaler
standardscaler = StandardScaler()
X = standardscaler.fit_transform(X)
print ("Pour vérifier que les données ont bien été transformées")
print (pd.DataFrame(X).head())


#création d'un jeu de test et d'apprentissage (70,30)
from sklearn.model_selection import train_test_split
validation_size=0.3 #30% du jeu de données pour le test
testsize = (1-validation_size)
seed = 30
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size=validation_size,
                                                    random_state=seed,
                                                    test_size=testsize)